### Data Exploration

In [ ]:
url = 'https://api.data.gov.sg/v1/environment/air-temperature?date=2022-01-16'
response = requests.get(url)

In [ ]:
rawdata = json.loads(response.content)

In [ ]:
metadata = rawdata['metadata']
items = rawdata['items']
api_info = rawdata['api_info']

In [ ]:
metadatadf = pd.DataFrame(metadata)
itemsdf = pd.DataFrame(items)
api_info # {'status':'healthy'}

{'status': 'healthy'}

In [ ]:
itemsdf[itemsdf.index%30==0]

### getWeatherInfo

In [ ]:
import requests
import time
import json
import pandas as pd
from datetime import datetime

In [ ]:
class getWeatherInfo_byTime:
  def __init__(self,start_date,end_date):
    self.intro()
    self.datetimeArray = []
    self.datetimeStringArray = []
    self.temperatureInfo = pd.DataFrame(columns=['datetime','period','AVG_temperature'])
    self.temperature_url = 'https://api.data.gov.sg/v1/environment/air-temperature?date_time='
    self.start_date = start_date
    self.end_date = end_date

  def intro(self):
    print('this class retrieves weather information specifically for DR programme')
    print('you can use getTemperature method and getRainfall method to explore more')
    print('make sure the start date and end date argument are in YYYY/MM/DD format')

  def getTemperature(self):
    start_stamp = int(datetime.timestamp(datetime.strptime(self.start_date,"%Y/%m/%d")))
    end_stamp = int(datetime.timestamp(datetime.strptime(self.end_date,"%Y/%m/%d")))

    for i in range(start_stamp,end_stamp,1800):
      datetime_sample = datetime.fromtimestamp(i)
      datetimeString_sample = datetime_sample.strftime("%Y-%m-%dT%H:%M:%S").replace(':','%3A')
      self.datetimeArray.append(datetime_sample)
      self.datetimeStringArray.append(datetimeString_sample)
    
    print('datetimeArray ready! There are {} entries in the array'.format(len(self.datetimeArray)))

    for i,item in enumerate(datetimeStringArray):
      url = self.temperature_url + item
      response = requests.get(url)
      mean_temperature = pd.DataFrame(json.loads(response.content)['items'][0]['readings']).value.mean()
      period = i%48 + 1
      self.temperatureInfo = self.temperatureInfo.append({'datetime':datetimeArray[i],'period':period,'AVG_temperature':mean_temperature},ignore_index=True)
      if period == 48:
        print('{} days data ready'.format((i+1)//48))


In [ ]:
class getWeatherInfo_byDate:
  def __init__(self,start_date,end_date):
    self.intro()
    self.dateArray = []
    self.dateStringArray = []
    self.temperatureInfo = pd.DataFrame()
    self.temperature_url = 'https://api.data.gov.sg/v1/environment/air-temperature?date='
    self.start_date = start_date
    self.end_date = end_date

  def intro(self):
    print('this class retrieves weather information specifically for DR programme')
    print('you can use getTemperature method and getRainfall method to explore more')
    print('make sure the start date and end date argument are in YYYY-MM-DD format')

  def getTemperature(self):
    start_stamp = int(datetime.timestamp(datetime.strptime(self.start_date,"%Y-%m-%d")))
    end_stamp = int(datetime.timestamp(datetime.strptime(self.end_date,"%Y-%m-%d")))

    for i in range(start_stamp,end_stamp,86400):
      datetime_sample = datetime.fromtimestamp(i)
      datetimeString_sample = datetime_sample.strftime("%Y-%m-%d")
      self.dateArray.append(datetime_sample)
      self.dateStringArray.append(datetimeString_sample)
    
    print('datetimeArray ready! There are {} days in the array'.format(len(self.dateArray)))

    for i,date in enumerate(self.dateStringArray):
      url = self.temperature_url + date
      response = requests.get(url)
      items_df = pd.DataFrame(json.loads(response.content)['items'])
      temp = items_df[items_df.index%30==0]
      periods = [i for i in range(1,49)]
      mean_temperatures = [pd.DataFrame(s).value.mean() for s in temp.readings]
      try:
        day_df = pd.DataFrame({'timestamp':temp.timestamp,'period':periods,'AVG_temperature':mean_temperatures})
        self.temperatureInfo = self.temperatureInfo.append(day_df, ignore_index=True)
        print('{} days ready!'.format(i+1))
      except Exception as e:
        print('the {}th days not ready! '.format(i+1),e)
        print('It is not ok for ' + date)
      

In [ ]:
DR_temperature = getWeatherInfo_byDate('2022-01-15','2022-03-01')

this class retrieves weather information specifically for DR programme
you can use getTemperature method and getRainfall method to explore more
make sure the start date and end date argument are in YYYY-MM-DD format


In [ ]:
DR_temperature.getTemperature()

datetimeArray ready! There are 45 days in the array
1 days ready!
the 2th days not ready!  All arrays must be of the same length
It is not ok for 2022-01-16
3 days ready!
4 days ready!
5 days ready!
6 days ready!
7 days ready!
8 days ready!
9 days ready!
10 days ready!
11 days ready!
12 days ready!
13 days ready!
14 days ready!
15 days ready!
16 days ready!
17 days ready!
18 days ready!
19 days ready!
20 days ready!
21 days ready!
22 days ready!
23 days ready!
24 days ready!
25 days ready!
26 days ready!
27 days ready!
28 days ready!
29 days ready!
30 days ready!
31 days ready!
32 days ready!
33 days ready!
34 days ready!
35 days ready!
36 days ready!
37 days ready!
38 days ready!
39 days ready!
the 40th days not ready!  All arrays must be of the same length
It is not ok for 2022-02-23
the 41th days not ready!  All arrays must be of the same length
It is not ok for 2022-02-24
42 days ready!
43 days ready!
44 days ready!
45 days ready!


In [ ]:
DR_temperature.temperatureInfo

,timestamp,period,AVG_temperature
0,2022-01-15T00:01:00+08:00,1,26.100000
1,2022-01-15T00:31:00+08:00,2,26.064286
2,2022-01-15T01:01:00+08:00,3,25.964286
3,2022-01-15T01:31:00+08:00,4,25.900000
4,2022-01-15T02:01:00+08:00,5,25.807143
...,...,...,...
2011,2022-02-28T21:32:00+08:00,44,27.007692
2012,2022-02-28T22:02:00+08:00,45,26.938462
2013,2022-02-28T22:32:00+08:00,46,26.838462
2014,2022-02-28T23:02:00+08:00,47,26.761538
